In [70]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

# Scrape all ad texts from urls

In [47]:
# import the data
df = pd.read_csv('Texas2021.csv')
df = pd.DataFrame(df)
df.head()

,ad_id,ad_url,ad_type,regions,advertiser_id,advertiser_name,ad_campaigns_list,date_range_start,date_range_end,num_of_days,...,spend_range_min_pln,spend_range_max_pln,spend_range_min_ron,spend_range_max_ron,spend_range_min_sek,spend_range_max_sek,spend_range_min_gbp,spend_range_max_gbp,spend_range_min_nzd,spend_range_max_nzd
0,CR134968385204125696,https://transparencyreport.google.com/politica...,Text,US,AR465982957815857152,LYDIA BEAN FOR CONGRESS,NaN,2021-03-11,2021-05-01,49,...,100,1000.0,750,37500.0,750,7500.0,50,500.0,200,1500.0
1,CR349206645328314368,https://transparencyreport.google.com/politica...,Image,US,AR221691299693068288,SAHAK NALBANDYAN,NaN,2022-01-12,2022-02-02,22,...,100,1000.0,75,750.0,750,7500.0,50,500.0,0,200.0
2,CR531379229376380928,https://transparencyreport.google.com/politica...,Text,US,AR279687823638921216,Glenn A. Hegar,NaN,2022-01-05,2022-03-01,47,...,100,1000.0,75,750.0,750,7500.0,50,500.0,0,200.0
3,CR133689996778405888,https://transparencyreport.google.com/politica...,Image,US,AR140325824329940992,Chad Crow,NaN,2022-02-17,2022-03-01,13,...,100,1000.0,75,750.0,750,7500.0,50,500.0,0,200.0
4,CR118816009636282368,https://transparencyreport.google.com/politica...,Text,US,AR393003423278039040,"CAITLYN JENNER FOR GOVERNOR 2021, INC.",NaN,2021-05-03,2021-05-26,17,...,100,1000.0,750,37500.0,750,7500.0,50,500.0,200,1500.0


In [48]:
# create a list of urls from text ads
urls = df[df['ad_type']=='Text']['ad_url'].to_list()
urls[1:10]

['https://transparencyreport.google.com/political-ads/library/advertiser/AR279687823638921216/creative/CR531379229376380928',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR393003423278039040/creative/CR118816009636282368',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR393003423278039040/creative/CR134265247518162944',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR67037258305241088/creative/CR502092019183648768',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR465982957815857152/creative/CR12089164707135488',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR122301255337902080/creative/CR528988616219688960',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR389693240443404288/creative/CR496800138438639616',
 'https://transparencyreport.google.com/political-ads/library/advertiser/AR19054570869096448/creative/CR8196477791974195

In [49]:
# chech the number of urls
len(urls)

654

In [50]:
# Google uses a secret API to get the ad texts from ad reports and import them into the ads pages
# use entity_id and creative_id to get url for each report
report_urls = []
for url in urls:
    entity_id = url.split('/')[-3]
    creative_id = url.split('/')[-1]
    report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
    report_urls.append(report_url)
report_urls[1:10]

['https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR279687823638921216&creative_id=CR531379229376380928&hl=en',
 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR393003423278039040&creative_id=CR118816009636282368&hl=en',
 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR393003423278039040&creative_id=CR134265247518162944&hl=en',
 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR67037258305241088&creative_id=CR502092019183648768&hl=en',
 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR465982957815857152&creative_id=CR12089164707135488&hl=en',
 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR122301255337902080&creative_id=CR528988616219688

In [51]:
# check if the number of the reports matches the number of urls
len(report_urls)

654

In [52]:
# now, let's scrape the ads text from each report
def get_text(report_url):
    response = requests.get(report_url)
    text = response.text.split('"]')[0].split('[')[-1]
    return text

In [53]:
ad_text = []
for report_url in report_urls:
    text = get_text(report_url)
    ad_text.append(text)

In [54]:
ad_text[1:10]

['"Glenn Hegar is a sixth generation Texan running for Comptroller. Learn more here!',
 '"Caitlyn Is The Change California Needs. Donate Now. Don\'t Wait!","No Time To Waste. Donate To Caitlyn Jenner For Governor.',
 '"Get Rid Of Gavin Once And For All. Caitlyn Has A Plan To Save California. Donate Today!","It\'s Official! Newsom Will Be Recalled. Caitlyn Will Put People Over Politics. Chip In.',
 '"Will we join the left to take America backwards, or will this GOP lead America forward?","If we choose the right answer, our country\'s future is unlimited. Join Rep. Kinzinger now!',
 '"Lydia stands for quality schools, more jobs, and protecting us from the climate crisis.","In Washington, she will work across the aisle to give every Texan a fair shot.',
 '"The filibuster stands in the way. We need to fix or nix the filibuster. You can help.',
 '"This campaign is about what we can achieve when we all come together.","Help define the future of our great state. Join us by making a donation t

In [104]:
df_texas = pd.DataFrame(ad_text)

In [105]:
df_texas.rename(columns = {0:'text'},inplace = True)

# Clean the text ads

In [106]:
df_texas

,text
0,"""Lydia has always stood for what's right, just..."
1,"""Glenn Hegar is a sixth generation Texan runni..."
2,"""Caitlyn Is The Change California Needs. Donat..."
3,"""Get Rid Of Gavin Once And For All. Caitlyn Ha..."
4,"""Will we join the left to take America backwar..."
...,...
649,"""California Deserves Better. California Deserv..."
650,"""Newsom Recall Is Official! Caitlyn Can Fix Ca..."
651,"""Newsom Recall Is Official! Caitlyn Can Fix Ca..."
652,"""Don't Let Chuck Schumer and Joe Biden Impleme..."


In [107]:
# check the dataset 
if df_texas['text'].isnull().sum():
    print (df_texas['text'].isnull().sum())
    df_texas = df_texas[df_texas['text'].notnull()]
    df_texas.reset_index(drop=True,inplace=True)

In [108]:
# function that filter the urls and symbols in the text 
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x) # filter symbols
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split()] 
    
    return ' '.join(ls)

In [109]:
df_texas['text'] = df_texas['text'].astype(str).apply(lambda x: filter_text(x))
df_texas

,text
0,lydia has always stood for what s right just a...
1,glenn hegar is a sixth generation texan runnin...
2,caitlyn is the change california needs donate ...
3,get rid of gavin once and for all caitlyn has ...
4,will we join the left to take america backward...
...,...
649,california deserves better california deserves...
650,newsom recall is official caitlyn can fix cali...
651,newsom recall is official caitlyn can fix cali...
652,don t let chuck schumer and joe biden implemen...


In [111]:
# check if there are duplicated texts, if yes, print them out
df_texas['text'].value_counts().head(20)

In [116]:
df_texas['text'].value_counts().head(20)

ca has had enough of newsom caitlyn can take back our state donate now help take back california newsom is a disaster donate to make ca red                                      19
newsom recall is official caitlyn can fix california donate now did you see the news gavin newsom will be recalled caitlyn can fix ca chip in 5                                  17
get rid of gavin once and for all caitlyn has a plan to save california donate today it s official newsom will be recalled caitlyn will put people over politics chip in         13
californians deserve leadership who is accountable to them caitlyn will be chip in now caitlyn will fight for you she is an outsider and a proven winner donate now              12
caitlyn is an outsider sacramento special interests have no power over her donate now drain the ca swamp newsom is controlled by special interests caitlyn is not donate now     12
fed up with one party leadership in ca chip in to fight for everyday californians gavin newsom doesn

In [91]:
# remove duplicated rows
df_texas.drop_duplicates(keep = 'first', inplace = True)
df_texas.reset_index(inplace = True,drop = True)

In [93]:
df_texas

,text
0,lydia has always stood for what s right just a...
1,glenn hegar is a sixth generation texan runnin...
2,caitlyn is the change california needs donate ...
3,get rid of gavin once and for all caitlyn has ...
4,will we join the left to take america backward...
...,...
296,we have to end the biden harris border crisis ...
297,washington changed lizzie fletcher tell lizzie...
298,glenn hegar a true conservative defending fait...
299,donate to the gop today to help us out raise t...
